# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **Berlin**, Germany.

Since there are lots of restaurants in Berlin we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Berlin center will be obtained using **Google Maps API geocoding** of well known Berlin location (Alexanderplatz)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Berlin city center.

Let's first find the latitude & longitude of Berlin city center, using specific, well known address and Google Maps geocoding API.

In [1]:
import folium
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import geocoder
import pgeocode

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [38]:
# read csv file
df = pd.read_csv('data/DE.txt', delimiter="\t", names=['country-code', 'postal-code', 'city',
                 'state', "state-abbrev", "region", "r-code", "province", "p-code", "latitude", "longitude", "accuracy"], encoding='utf-8')

def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values) == True]

# drop rows where province does not include Rostock
df = filter_rows_by_values(df, "province", ["Kreisfreie Stadt Rostock", "Landkreis Rostock"])

df.head()

# df.shape


,country-code,postal-code,city,state,state-abbrev,region,r-code,province,p-code,latitude,longitude,accuracy
6251,DE,17166,Groß Wokern,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.7513,12.4925,4.0
6252,DE,17166,Bristow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.7123,12.6257,4.0
6253,DE,17166,Dalkendorf,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.8148,12.5021,4.0
6254,DE,17166,Bülow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.6899,12.6007,4.0
6255,DE,17166,Alt Sührkow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.8024,12.6512,4.0


In [33]:
# get Rostock geo-location
address = 'Rostock, Germany'
geolocator = Nominatim(user_agent="coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(
    f"The geograpical coordinates of Rostock Germnay are {latitude}, {longitude}.")


The geograpical coordinates of Rostock Germnay are 54.0924445, 12.1286127.


In [34]:
# create map
map_rostock = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, city in zip(df['latitude'], df['longitude'], df['city']):
    label = f"{city}"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_rostock)
map_rostock


In [ ]:
# get the geo-location of each post-code
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis(
            f"{postal_code}, Rostock, Germnay")
        lati_long_coords = g.latlng
    return lati_long_coords


# test
get_latilong('18069')


In [ ]:
from constants import FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET
# Foursquare API config
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print(f"CLIENT_ID: {FOURSQUARE_CLIENT_ID}")
print(f"CLIENT_SECRET: {FOURSQUARE_CLIENT_SECRET}")


Your credentails:
CLIENT_ID: MZUGNTHUN4CZJ1BURHWGXXXO0XYGMRNDRWB2KADTMQYBKYHZ
CLIENT_SECRET: MTQVYA102ORA4YZVXD5J1Z200IZMAFMRVKRP4AESXUO0G0K5
